Importing Data
---------------------

This notebook shows how to conveniently input your data into ComPWA using the python interface!
Also it shows how to calculate kinematic variables.

In [ ]:
import pycompwa.ui as pwa
import uproot
import numpy as np

Before we can start, a dummy root data file has to be created. Normally you would skip the next step, since you already have a data file that you want to import.

In [ ]:
with uproot.recreate("tuples.root") as f:
    f["data"] = uproot.newtree({"D0.px": "float64",
                                "D0.py": "float64",
                                "D0.pz": "float64",
                                "D0.E": "float64",
                                "Dm.px": "float64",
                                "Dm.py": "float64",
                                "Dm.pz": "float64",
                                "Dm.E": "float64",
                                "pip.px": "float64",
                                "pip.py": "float64",
                                "pip.pz": "float64",
                                "pip.E": "float64"
                               })
    f["data"].extend({"D0.px": np.array([-0.63207397,  0.11329174, -0.32988257]),
                      "D0.py": np.array([-0.02727499,  0.39965045, 0.1848439 ]),
                      "D0.pz": np.array([ 0.16475981,  0.28701658, -0.37938268]),
                      "D0.E": np.array([2.0180088, 1.92316975, 1.93920292]),
                      "Dm.px": np.array([0.48782456, 0.08336758, 0.10971082]),
                      "Dm.py": np.array([-0.10812153, -0.56649819, -0.13254077]),
                      "Dm.pz": np.array([-0.12110724, -0.35891476, 0.44891566]),
                      "Dm.E": np.array([1.95351745, 1.96315494, 1.94968155]),
                      "pip.px": np.array([ 0.16118203, -0.10011399, 0.24527202]),
                      "pip.py": np.array([ 0.10152041, 0.19745644, -0.04717316]),
                      "pip.pz": np.array([-0.03064159, 0.0802533, -0.04723599]),
                      "pip.E": np.array([0.23812766, 0.27373762, 0.28999094])
                     })

First load the data from the root file and bundle the 4 vector data in numpy arrays

In [ ]:
tree = uproot.open("tuples.root")["data"]
parts = tree.arrays(['D0*', 'Dm*', 'pip*'])
D0s = np.array((parts[b'D0.px'],parts[b'D0.py'],parts[b'D0.pz'],parts[b'D0.E'])).T
Dms = np.array((parts[b'Dm.px'],parts[b'Dm.py'],parts[b'Dm.pz'],parts[b'Dm.E'])).T
pis = np.array((parts[b'pip.px'],parts[b'pip.py'],parts[b'pip.pz'],parts[b'pip.E'])).T

Now construct ComPWA `Event`s by handing it a list of `FourMomentum`.

In [ ]:
evts = [pwa.Event(pwa.FourMomentumList([pwa.FourMomentum(x[0]), pwa.FourMomentum(x[1]), pwa.FourMomentum(x[2])]))
     for x in zip(D0s, Dms, pis)]

evtcoll = pwa.EventCollection(
     [421, -411, 211],
     pwa.EventList(evts)
)
print("Event 0:",evtcoll.events[0].four_momenta)

In [ ]:
partlist = pwa.read_particles('../../ComPWA/Physics/particle_list.xml')
kin = pwa.HelicityKinematics(partlist, [421], [421, -411, 211])
kin.create_all_subsystems()

Using the `Kinematics.convert()` function the `EventCollection` can be converted to a `DataSet`, which contains all of the registered kinematic variables (in this example all possible kinematic variables).

In [ ]:
dataset = kin.convert(evtcoll)

In [ ]:
print("kinematic variable names:", list(dataset.data.keys()))
print(dataset.data)